In [1]:
import os
import pickle
import numpy as np
from scipy import interp
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.svm import SVC

In [2]:
CHANNELS = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FCz', 'FC6', 'T7', 'C3', 'Cz', 'C4',
            'T8', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']

Define method for grid search with different classification methods to find their optimal parameters

In [6]:
def perform_logistic_regression(X, y, n_splits, score):
    # Grid search with logistic regression
    lr = LogisticRegression(C=1.0)

    # GridSearch for liblinear solver
    C_range = [0.1, 1, 10, 100, 1000, 10000]
    penalty_options = ['l1', 'l2']
    fit_intercept_options = [True, False]
    solver_option = ['liblinear']
    class_weight_option = ['auto', '']

    param_grid = dict(C=C_range, penalty=penalty_options, fit_intercept=fit_intercept_options, solver=solver_option)

    # solver_options = ['newton-cg', 'lbfgs', 'sag', 'saga']
    # multi_class_options = ['ovr', 'multinomial']
    # class_weight_options = [None, 'balanced']

    # param_grid = dict(solver = solver_options, multi_class = multi_class_options, class_weight = class_weight_options)

    best_params = perform_grid_search(lr, X, y, score, n_splits, param_grid, "Logistic Regression")
    
    return best_params
    

def perform_lda(X, y, n_splits, score):
    lda = LinearDiscriminantAnalysis()

    solver_options = ['lsqr']
    #shrinkage_options = [None, 'auto']
    shrinkage = np.arange(0.0, 1.05, 0.05)
    
    param_grid = dict(solver=solver_options, shrinkage=shrinkage)

    best_params = perform_grid_search(lda, X, y, score, n_splits, param_grid, "LDA")
    
    return best_params
    
    
    
def perform_grid_search(clf, X, y, score, n_splits, param_grid, name, verbose=False):
    
    cv = StratifiedKFold(n_splits=10)
    
    grid_clf = GridSearchCV(clf, param_grid, cv=cv, scoring=score)
    grid_clf.fit(X, y)
    
    if verbose:
        print("# Tuning parameters for %s" % name)
        print()
        # best results
        print("The best parameters are %s with a score of %0.2f" \
              % (grid_clf.best_params_, grid_clf.best_score_))
        print()
        # complete results
        means = grid_clf.cv_results_['mean_test_score']
        stds = grid_clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, grid_clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print("\n")
    
    return grid_clf.best_params_

In [7]:
def run_evaluation(clf, X, y, n_splits, score='accuracy'):
    
    results = []
    
    for iteration in range(0, 10):
        
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=iteration)

        # Variable for accuracy score
        mean_score = 0
        
        for train, test in cv.split(X, y):
            
            # Compute accuracy score
            y_pred = clf.fit(X[train], y[train]).predict(X[test])
            
            if score is 'accuracy':
                mean_score += accuracy_score(y[test], y_pred)
            elif score is 'precision':
                mean_score += precision_score(y[test], y_pred, pos_label=0)
            elif score is 'recall':
                mean_score += recall_score(y[test], y_pred, pos_label=0)

        # Mean accuracy
        mean_score /= n_splits
        
        results.append(mean_score)
            
    return results

In [24]:
def run_set_of_evaluations(datasets, n_splits, optimize_score, evaluate_scores, name):
    
    # initialize result dictionary and lists for each score
    results = {}
    
    for score in evaluate_scores:
        results[score] = []
    
    # evaluate each dataset
    for dataset in datasets:
        
        X, y = dataset.get_features_and_labels('Consc_Group')
        
        # parameter optimization
        best_params_lda = perform_lda(X, y, n_splits, optimize_score)
        best_params_lr = perform_logistic_regression(X, y, n_splits, optimize_score)

        lda = LinearDiscriminantAnalysis(**best_params_lda)
        lr = LogisticRegression(**best_params_lr)
        
        # evaluate best parameters on each score
        for score in evaluate_scores:

            results_dataset = []

            results_lda = run_evaluation(lda, X, y, n_splits, score)
            results_lr = run_evaluation(lr, X, y, n_splits, score)

            results_dataset.append(results_lda)
            results_dataset.append(results_lr)

            results[score].append(results_dataset)

    # result presentation
    
    for score in evaluate_scores:
        
        results[score] = np.array(results[score])
        
        means = np.mean(results[score], axis=2)
        stds = np.std(results[score], axis=2)

        print('%i, %s, %s:' % (n_splits, optimize_score, score))
        print(means[:,0])
        print(means[:,1])
        
        ind = np.arange(len(means))
        width = 0.35

        fig, ax = plt.subplots()
        rects1 = ax.bar(ind - width/2, means[:,0], width, yerr=stds[:,0], color='SkyBlue', label='LDA', capsize=5)
        rects2 = ax.bar(ind + width/2, means[:,1], width, yerr=stds[:,1], color='IndianRed', label='LR', capsize=5)

        ax.set_ylabel(score)
        ax.set_title('%s by instruction condition and classifier' % score)
        ax.set_xticks(ind)
        ax.set_xticklabels(('STD comp', 'STD incomp', 'TS comp', 'TS incomp'))
        ax.legend()

        plot_path = os.path.abspath(os.path.join(image_directory, name + '_' + score + '_lr_lda.png'))

        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        
    

### Evaluation of IntErr2 data

Set important directories.

In [25]:
working_directory = 'L:/int_err2'
current_path = os.path.abspath(working_directory)
image_directory = os.path.abspath(os.path.join(current_path, 'images'))
pickle_directory = os.path.abspath(os.path.join(current_path, 'pickles'))

Load separability indices.

In [26]:
std_comp_corr_file = os.path.abspath(os.path.join(pickle_directory, 'std_comp_corr.pkl'))
with open(std_comp_corr_file, 'rb') as pickle_file:
    std_comp_corr = pickle.load(pickle_file)
    
std_incomp_corr_file = os.path.abspath(os.path.join(pickle_directory, 'std_incomp_corr.pkl'))
with open(std_incomp_corr_file, 'rb') as pickle_file:
    std_incomp_corr = pickle.load(pickle_file)
    
ts_comp_corr_file = os.path.abspath(os.path.join(pickle_directory, 'ts_comp_corr.pkl'))
with open(ts_comp_corr_file, 'rb') as pickle_file:
    ts_comp_corr = pickle.load(pickle_file)
    
ts_incomp_corr_file = os.path.abspath(os.path.join(pickle_directory, 'ts_incomp_corr.pkl'))
with open(ts_incomp_corr_file, 'rb') as pickle_file:
    ts_incomp_corr = pickle.load(pickle_file)

In [27]:
std_comp_corr.extract_features([[112, 160]], CHANNELS, 'Consc_Group')
std_incomp_corr.extract_features([[-60, -40], [112, 148]], CHANNELS, 'Consc_Group')
ts_comp_corr.extract_features([[112, 152], [468, 484]], CHANNELS, 'Consc_Group')
ts_incomp_corr.extract_features([[348, 380]], CHANNELS, 'Consc_Group')
# ts_incomp_corr.extract_features([[-20, 0], [348, 380]], CHANNELS, 'Consc_Group')

{1: LastUpdatedOrderedDict([('ID', 1),
                         ('Consc_Group', '2'),
                         ('Fp1_112.0-120.0', 1.1028888888888888),
                         ('Fp2_112.0-120.0', 1.5188333333333335),
                         ('F7_112.0-120.0', 2.7699666666666665),
                         ('F3_112.0-120.0', 3.9848777777777777),
                         ('Fz_112.0-120.0', 3.8530444444444445),
                         ('F4_112.0-120.0', 1.7692),
                         ('F8_112.0-120.0', -1.1903555555555558),
                         ('FC5_112.0-120.0', 2.2287111111111115),
                         ('FCz_112.0-120.0', 4.727177777777778),
                         ('FC6_112.0-120.0', -0.2754777777777778),
                         ('T7_112.0-120.0', -0.8401888888888889),
                         ('C3_112.0-120.0', 3.3061),
                         ('Cz_112.0-120.0', 2.4052333333333333),
                         ('C4_112.0-120.0', 0.0862222222222222),
                     

In [32]:
run_set_of_evaluations([std_comp_corr, std_incomp_corr, ts_comp_corr, ts_incomp_corr], 10, 'accuracy', ['accuracy', 'precision', 'recall'], 'interr2')

4, accuracy, accuracy:
[0.63832418 0.61662088 0.6364011  0.60961538]
[0.63241758 0.52211538 0.69464286 0.50274725]
4, accuracy, precision:
[0.5952877  0.57273268 0.6147619  0.57369048]
[0.59930556 0.46766865 0.66956349 0.46458333]
4, accuracy, recall:
[0.56666667 0.57916667 0.54583333 0.52083333]
[0.48333333 0.5        0.65833333 0.45833333]
5, accuracy, accuracy:
[0.64127273 0.608      0.65654545 0.61327273]
[0.63727273 0.50018182 0.66490909 0.53018182]
5, accuracy, precision:
[0.60647619 0.55434127 0.64261111 0.6092381 ]
[0.63061905 0.42961111 0.62709524 0.4735    ]
5, accuracy, recall:
[0.565 0.562 0.584 0.518]
[0.488 0.492 0.657 0.47 ]


C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


6, accuracy, accuracy:
[0.65740741 0.6462963  0.65925926 0.60925926]
[0.64814815 0.51296296 0.68703704 0.52777778]
6, accuracy, precision:
[0.62202381 0.60464286 0.66638889 0.59541667]
[0.63611111 0.41716931 0.6572619  0.48011905]
6, accuracy, recall:
[0.59166667 0.62916667 0.55833333 0.5125    ]
[0.4875     0.45       0.67083333 0.49583333]


C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


7, accuracy, accuracy:
[0.67585034 0.6390873  0.65980726 0.59659864]
[0.65873016 0.48353175 0.68871882 0.50022676]
7, accuracy, precision:
[0.6297619  0.60102041 0.675      0.56607143]
[0.63428571 0.40908163 0.64942177 0.44462585]
7, accuracy, recall:
[0.63214286 0.625      0.56190476 0.49642857]
[0.5        0.47738095 0.68571429 0.46785714]


...

C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


6, recall, accuracy:
[0.65740741 0.62962963 0.64814815 0.61666667]
[0.64814815 0.54444444 0.68703704 0.52777778]


C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


6, recall, precision:
[0.62202381 0.58388889 0.6375     0.6056746 ]
[0.63611111 0.44833333 0.6572619  0.47019841]
6, recall, recall:
[0.59166667 0.60833333 0.5125     0.50416667]
[0.4875     0.27083333 0.67083333 0.5       ]


KeyboardInterrupt: 

### Evaluation of IntErr1 data

Set important directories.

In [14]:
working_directory = 'L:/int_err'
current_path = os.path.abspath(working_directory)
image_directory = os.path.abspath(os.path.join(current_path, 'images'))
pickle_directory = os.path.abspath(os.path.join(current_path, 'pickles'))

Load separability indices.

In [15]:
consc_std_corr_file = os.path.abspath(os.path.join(pickle_directory, 'consc_std_corr.pkl'))
with open(consc_std_corr_file, 'rb') as pickle_file:
    consc_std_corr = pickle.load(pickle_file)
    
consc_rv_corr_file = os.path.abspath(os.path.join(pickle_directory, 'consc_rv_corr.pkl'))
with open(consc_rv_corr_file, 'rb') as pickle_file:
    consc_rv_corr = pickle.load(pickle_file)  

consc_rr_corr_file = os.path.abspath(os.path.join(pickle_directory, 'consc_rr_corr.pkl'))
with open(consc_rr_corr_file, 'rb') as pickle_file:
    consc_rr_corr = pickle.load(pickle_file)  

In [17]:
consc_std_corr.extract_features([[124, 188], [192, 212], [246, 278]], CHANNELS, 'Consc_Group')
# [160, 180], [204, 240] - [120, 192], [200, 220]
consc_rv_corr.extract_features([[120, 192], [200, 220]], CHANNELS, 'Consc_Group')
# [160, 180], [204, 240] - [112, 168], [172, 212]
consc_rr_corr.extract_features([[112, 168], [172, 212]], CHANNELS, 'Consc_Group')

{1: LastUpdatedOrderedDict([('ID', 1),
                         ('Consc_Group', '2'),
                         ('Fp1_53.0-67.0', -1.6231133333333336),
                         ('Fp2_53.0-67.0', -1.2786000000000002),
                         ('F7_53.0-67.0', 0.7447466666666666),
                         ('F3_53.0-67.0', -1.7042800000000002),
                         ('Fz_53.0-67.0', -1.7382733333333336),
                         ('F4_53.0-67.0', -1.083613333333333),
                         ('F8_53.0-67.0', -0.21118),
                         ('FC5_53.0-67.0', -1.2143733333333333),
                         ('FCz_53.0-67.0', -1.8648266666666666),
                         ('FC6_53.0-67.0', 0.37378666666666666),
                         ('T7_53.0-67.0', 0.3446733333333334),
                         ('C3_53.0-67.0', 0.8181133333333334),
                         ('Cz_53.0-67.0', -1.4963600000000001),
                         ('C4_53.0-67.0', 0.5070733333333333),
                         ('T8

In [19]:
for optimize_score in ['accuracy', 'precision', 'recall']:
    for i in range(4, 10):
        run_set_of_evaluations([consc_std_corr, consc_rv_corr, consc_rr_corr], i, optimize_score, ['accuracy', 'precision', 'recall'], 'interr1')

C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mike\AppData\Local\conda\conda\envs\seegnature\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


10, accuracy, accuracy:
[[0.645      0.7095    ]
 [0.724      0.70516667]
 [0.66166667 0.68483333]]
10, accuracy, precision:
[[0.60666667 0.68833333]
 [0.72833333 0.7335    ]
 [0.67216667 0.70583333]]
10, accuracy, recall:
[[0.595      0.635     ]
 [0.7        0.68      ]
 [0.64333333 0.68666667]]
